In [1]:
import os
import lightgbm as lgb
import pandas as pd
import warnings
import numpy as np
import random
import ast


# importo los .py
import sys
sys.path.append("auxiliares")
import config
import transformaciones
import lightgbm_aux
import extras

pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)

# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/Users/herna/labo3_empresa3_repo/")
#os.chdir("C:/diego_tools/labo3/")
os.chdir("/home/dcastro_contacto/buckets/b1/")
####################################################

In [3]:
arch_entrada = "datasets/emp3_sellout_producto_transf_fe.csv"

arch_min_max_prod = "datasets/emp3_min_max_prod.csv"
arch_mean_std_prod = "datasets/emp3_mean_std_prod.csv"

carpeta_base_exp = "exp/intermedia_2/"

In [4]:
# Se genera la prediccion para el indice elegido de la optimizacion bayesiana
FUTURE_OPT_BAY_INDEX = 3
EXP_NOMBRE = "20231217_165826"

In [5]:
GLOBAL_PRODUCT_IDS = pd.DataFrame() #aca solamente se define, se setea mas abajo

In [6]:
def destransformar_vector(prod_values,y_vector):
    # Dataframes auxiliares
    if config.TIPO_TRANSF_PARAM == "normalizacion":
        print("Normalizacion")
        df_prod_val1_val2 = pd.read_csv(arch_min_max_prod)
    
    elif config.TIPO_TRANSF_PARAM == "estandarizacion":
        print("Estandarizacion")
        df_prod_val1_val2 = pd.read_csv(arch_mean_std_prod)
    
    df_calculo = pd.DataFrame(data={"product_id":prod_values})
    df_calculo = df_calculo.merge(df_prod_val1_val2,how="left",on="product_id")
    df_calculo = df_calculo.sort_values(by="product_id",ascending=True)
    
    df_calculo["y"] = y_vector
    df_calculo["y_destransformado"]=df_calculo.apply(lambda row: transformaciones.destransformar_valor(config.TIPO_TRANSF_PARAM,row["y"],row["valor_1"],row["valor_2"]),axis=1)

    return df_calculo.y_destransformado

In [7]:
##############################################

In [8]:
carpeta_exp = carpeta_base_exp + EXP_NOMBRE + "/"

In [9]:
df_sellout = pd.read_csv(arch_entrada)
df_sellout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34707 entries, 0 to 34706
Columns: 446 entries, product_id to cat3_tn_bollinger_band_lower_15
dtypes: bool(1), float64(437), int64(5), object(3)
memory usage: 117.9+ MB


In [10]:
df_prods_prediccion_future=pd.read_csv(config.ARCH_PRODUCTOS_PREDICCION_FUTURE)
df_prods_prediccion_future.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   product_id  780 non-null    int64
dtypes: int64(1)
memory usage: 6.2 KB


In [11]:
### Categoricas
df_sellout = extras.convertir_categoricas_prod(df_sellout)

In [12]:
# Modelo Final
cols_remover_entren = ["tn_mas_2","periodo"]

df_train_modelo_final = df_sellout[(df_sellout.periodo <= config.MODELO_FINAL_PERIODO_LIMITE_PARAM) & (df_sellout.periodo >= config.PERIODO_INICIO_PARAM)]
df_future = df_sellout[(df_sellout.product_id.isin(df_prods_prediccion_future.product_id)) & (df_sellout.periodo == config.FUTURE_PERIODO_PARAM)]
df_future.reset_index(inplace=True, drop=True)

print("Periodos entrenar MODELO FINAL:",df_train_modelo_final.periodo.unique())
print("Periodos FUTURE:",df_future.periodo.unique())

# Variables entrenamiento
X_train_modelo_final = df_train_modelo_final.drop(columns=cols_remover_entren, axis=1)
y_train_modelo_final = df_train_modelo_final.tn_mas_2
cols_entren = X_train_modelo_final.columns.tolist()

X_future = df_future.drop(columns=cols_remover_entren, axis=1)

print("\nX_train_all:", X_train_modelo_final.shape)
print("y_train_all:", y_train_modelo_final.shape)
print("\nX_future:", X_future.shape)

lgbtrain_modelo_final = lgb.Dataset(data=X_train_modelo_final, label=y_train_modelo_final, feature_name=cols_entren)

Periodos entrenar MODELO FINAL: [201701 201702 201703 201704 201705 201706 201707 201708 201709 201710
 201711 201712 201801 201802 201803 201804 201805 201806 201807 201808
 201809 201810 201811 201812 201901 201902 201903 201904 201905 201906
 201907 201909 201910]
Periodos FUTURE: [201912]

X_train_all: (32244, 444)
y_train_all: (32244,)

X_future: (780, 444)


In [13]:
df_opt_bay = pd.read_excel(carpeta_exp + "opt_bay.xlsx")

future_params = ast.literal_eval(df_opt_bay.iloc[FUTURE_OPT_BAY_INDEX].lgb_params) #convierto string a diccionario
future_best_iter = int(df_opt_bay.iloc[FUTURE_OPT_BAY_INDEX].best_iteration)
y_pred_future = lightgbm_aux.semillerio(lgbtrain_modelo_final, future_params, future_best_iter, X_future,config.CANT_SEMILLAS)

In [14]:
# Guardo predicciones
y_pred_future = extras.limpiar_prediccion(destransformar_vector(df_prods_prediccion_future.product_id,y_pred_future))

# Prediccion Future
df_pred = pd.DataFrame(data={"product_id":X_future.product_id,"tn":y_pred_future})

df_pred.to_csv(carpeta_exp + str(FUTURE_OPT_BAY_INDEX) + "_prediccion.csv",index=False)

Estandarizacion


In [15]:
df_pred

,product_id,tn
0,20001,1586.59
1,20002,1616.30
2,20003,900.04
3,20004,665.32
4,20005,619.17
...,...,...
775,21263,0.15
776,21265,0.19
777,21266,0.19
778,21267,0.19
